## ELT (Extracción, carga y transformación de datos)

#Ejercico de practica manip. de datos

El equipo de negocio te ha pedido que desde [esta API indicada](https://eodhistoricaldata.com/financial-apis/api-for-historical-data-and-volumes/), debes extraer los datos diarios del precio de las acciones de todas las compañías del S&P500, los cuales incluyen los precios de:

- Apertura
- Máximo
- Mínimo
- De cierre
- Cierre ajustado de la acción
- Volumen transado.

El formato que entrega la API es el siguiente:

Estos datos deben ser cargados en una base de datos para su consumo por parte de los Data Scientist que ejecutarán los análisis y visualizaciones para el negocio. Ellos han solicitado, que para ello dejes creadas algunas tablas con métricas pre-calculadas para cada una de las compañías del S&P500 :

- Precio más bajo: booleano, obtenga 1 si el precio de cierre es el precio mínimo para el período
- Precio más alto: booleano, obtenga 1 si el precio de cierre fue el precio más alto del período
- Precio máximo
- Precio percentil 75%
- Precio medio
- Precio promedio
- Coeficiente de variación: Calcular la relación entre la desviación estándar y el precio medio
- Precio percentil 25%
- Precio mínimo
- Cambio de precio como proporción: (Precio de cierre final-Precio de cierre inicial)/Precio de cierre inicial
- Ratio del precio de rango frente al precio de cierre: el rango del precio de la acción dividido por el precio de cierre medio del período.
- Volumen promedio del período

Todas estas otras métricas deben calcularse para este intervalo de tiempo:

- Durante los últimos 7 días atrás
- Durante las últimas 52 semanas
- Durante los últimos 5 años
- Semanal usando el formato: número de semana-año
- Anual (Para cada año que ha existido la empresa incluido el año actual)

**Entregables:**

1. Un diseño de lo que harás en un documento en Notion
2. El código terminado en tu Github personal
3. La documentación técnica mínima  (input, output y qué hace)

In [ ]:
import requests
import json
from getpass import getpass
import pandas as pd
from io import BytesIO
from io import StringIO

In [ ]:
desde=input("ingrese la fecha desde cuando desea obtener la data en formato YYYY-MM-DD")

ingrese la fecha desde cuando desea obtener la data en formato YYYY-MM-DD2023-02-08


In [ ]:
hasta=input("ingrese la fecha desde cuando desea obtener la data")

ingrese la fecha desde cuando desea obtener la data2023-02-15


In [ ]:
print(desde, "hasta", hasta)

2023-02-08 hasta 2023-02-15


In [ ]:
#Cargamos la data con requests.get
response = requests.get('https://eodhistoricaldata.com/api/eod/MCD.US?from='+desde+'&to='+hasta+'&period=d&api_token=63ecef84b77a71.97775831')

In [ ]:
response

<Response [200]>

In [ ]:
type(response.content) #al ver que la data viene en bytes, usamos la biblioteca StringIO para poder manipularla

bytes

In [ ]:
response.content #el contenido de la data viene en .content

b'Date,Open,High,Low,Close,Adjusted_close,Volume\n2023-02-08,265.09,265.89,262.45,262.82,262.82,2801800\n2023-02-09,263.69,263.95,260.19,260.66,260.66,3534600\n2023-02-10,260.87,262.69,260.29,262.42,262.42,3049900\n2023-02-13,263.7,266.79,263,266.2,266.2,2418300\n2023-02-14,266.65,267.52,264.59,266.61,266.61,2202100\n2023-02-15,266.27,267.68,265.15,266.56,266.56,1862700\n'

In [ ]:
s=str(response.content,'utf-8')

data = StringIO(s) 

df=pd.read_csv(data)

In [ ]:
df

,Date,Open,High,Low,Close,Adjusted_close,Volume
0,2023-02-08,265.09,265.89,262.45,262.82,262.82,2801800
1,2023-02-09,263.69,263.95,260.19,260.66,260.66,3534600
2,2023-02-10,260.87,262.69,260.29,262.42,262.42,3049900
3,2023-02-13,263.70,266.79,263.00,266.20,266.20,2418300
4,2023-02-14,266.65,267.52,264.59,266.61,266.61,2202100
5,2023-02-15,266.27,267.68,265.15,266.56,266.56,1862700


#Se asume que se necesita una columna con 0 si no es el minimo del periodo y un 1 si lo es, tanto para min como para max.

In [ ]:
min=df["Close"].min()
min

260.66

In [ ]:
for i in df.index:
  if df.loc[i,"Close"]==min:
    df.loc[i,"lowest_price"]=1
  else:
    df.loc[i,"lowest_price"]=0

In [ ]:
df

,Date,Open,High,Low,Close,Adjusted_close,Volume,lowest_price
0,2023-02-08,265.09,265.89,262.45,262.82,262.82,2801800,0.0
1,2023-02-09,263.69,263.95,260.19,260.66,260.66,3534600,1.0
2,2023-02-10,260.87,262.69,260.29,262.42,262.42,3049900,0.0
3,2023-02-13,263.70,266.79,263.00,266.20,266.20,2418300,0.0
4,2023-02-14,266.65,267.52,264.59,266.61,266.61,2202100,0.0
5,2023-02-15,266.27,267.68,265.15,266.56,266.56,1862700,0.0


In [ ]:
max=df["Close"].max()
max

266.61

In [ ]:
for i in df.index:
  if df.loc[i,"Close"]==max:
    df.loc[i,"highest_price"]=1
  else:
    df.loc[i,"highest_price"]=0

In [ ]:
df

,Date,Open,High,Low,Close,Adjusted_close,Volume,lowest_price,highest_price
0,2023-02-08,265.09,265.89,262.45,262.82,262.82,2801800,0.0,0.0
1,2023-02-09,263.69,263.95,260.19,260.66,260.66,3534600,1.0,0.0
2,2023-02-10,260.87,262.69,260.29,262.42,262.42,3049900,0.0,0.0
3,2023-02-13,263.70,266.79,263.00,266.20,266.20,2418300,0.0,0.0
4,2023-02-14,266.65,267.52,264.59,266.61,266.61,2202100,0.0,1.0
5,2023-02-15,266.27,267.68,265.15,266.56,266.56,1862700,0.0,0.0


In [ ]:
#

#Se asume que el valor de la columna High tiene los valores máximos y que no es necesario chequear las otras columnas, ya que no superarán la anteriomente nombrada, además de dejar estos datos en una variable junto con toda la otra información solicitada.

In [ ]:
highest_value = df["High"].max()
highest_value

267.68

In [ ]:
#Como no se define el porcentil .75 para un valor dado, se realiza el mismo para todas las colmnas (bviar para loswest_price y highest_price). 
#Se asume el metodo de interpolacion al más cercano
df.quantile(.75, interpolation="nearest")


Open                  266.27
High                  267.52
Low                   264.59
Close                 266.56
Adjusted_close        266.56
Volume            3049900.00
lowest_price            0.00
highest_price           0.00
Name: 0.75, dtype: float64

In [ ]:
#al igual que lo anterior, como no se establece para qué valores, se deja el valor de mediana y media para todos los datos
df.median()

<ipython-input-17-5ab130f88b1c>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.median()


Open                  264.395
High                  266.340
Low                   262.725
Close                 264.510
Adjusted_close        264.510
Volume            2610050.000
lowest_price            0.000
highest_price           0.000
dtype: float64

In [ ]:
df.mean()

<ipython-input-18-c61f0c8f89b5>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean()


Open              2.643783e+02
High              2.657533e+02
Low               2.626117e+02
Close             2.642117e+02
Adjusted_close    2.642117e+02
Volume            2.644900e+06
lowest_price      1.666667e-01
highest_price     1.666667e-01
dtype: float64

In [ ]:
df.std()

<ipython-input-19-ce97bb7eaef8>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.std()


Open                   2.120740
High                   2.027932
Low                    2.087510
Close                  2.568318
Adjusted_close         2.568318
Volume            606541.827082
lowest_price           0.408248
highest_price          0.408248
dtype: float64

In [ ]:
coef_dispersion =df.std()/df.mean()
coef_dispersion

<ipython-input-20-62cedb6619eb>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  coef_dispersion =df.std()/df.mean()


Open              0.008022
High              0.007631
Low               0.007949
Close             0.009721
Adjusted_close    0.009721
Volume            0.229325
lowest_price      2.449490
highest_price     2.449490
dtype: float64

In [ ]:
df.quantile(.25, interpolation="nearest")

Open                  263.69
High                  263.95
Low                   260.29
Close                 262.42
Adjusted_close        262.42
Volume            2202100.00
lowest_price            0.00
highest_price           0.00
Name: 0.25, dtype: float64

In [ ]:
#asumimos que el precio min está en la columna de minimos Min

min_value = df["Low"].min()
min_value

260.19

In [ ]:
#Cambio de precio como proporción: (Precio de cierre final-Precio de cierre inicial)/Precio de cierre inicial
df["prop_change"]=(df["Close"]-df["Open"])/df["Open"]


In [ ]:
df["prop_change"]

0   -0.008563
1   -0.011491
2    0.005942
3    0.009480
4   -0.000150
5    0.001089
Name: prop_change, dtype: float64

In [ ]:
#Ratio del precio de rango frente al precio de cierre: el rango del precio de la acción dividido por el precio de cierre medio del período

df["close_ratio"]=(df["Close"]-df["Open"])/df["Close"].mean()
df["close_ratio"]

0   -0.008592
1   -0.011468
2    0.005867
3    0.009462
4   -0.000151
5    0.001098
Name: close_ratio, dtype: float64

In [ ]:
#Volumen promedio del período

avg_vol= df["Volume"].mean()
avg_vol

2644900.0